In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
from hcplot import loadLibraries, Figure, Points, Line, mapping, single, grid, matrix, wrap, scales
from hcplot.scale import brewer
from hcplot.color import ColorBrewer, D3Colors

In [ ]:
loadLibraries()

In [ ]:
from ggplot import mpg, mtcars, diamonds

In [ ]:
%%sh 
cd /tmp
wget -q https://raw.githubusercontent.com/pandas-dev/pandas/master/pandas/tests/data/iris.csv

In [ ]:
import pandas as pd
iris = pd.read_csv('/tmp/iris.csv')

In [ ]:
fig1 = Figure(mpg, 
              mapping("displ", "hwy", color="cyl", shape="drv"),
              wrap(["class"], ncols=3, scales="free"),
              width=1280) \
     + Points(scales(color=brewer("qual", "Dark2")), size=4)
fig1

In [ ]:
fig2 = Figure(mpg, 
              mapping("displ", "cty"), 
              grid(["drv"], ["cyl"], labels=True, scales="fixed"), 
              width=1024)
fig2 + Points()
fig2

In [ ]:
fig3 = Figure(mtcars, 
              mapping("mpg", "wt"), 
              grid(["vs","am"], ["gear"], scales="fixed"), 
              width=800, ratio=0.75)
fig3 + Points(size=3)
fig3

In [ ]:
fig4 = Figure(iris, 
              mapping(["PetalLength", "PetalWidth", "SepalWidth", "SepalLength"], color="Name", shape="Name"),
              matrix(scales="free"),
              width=960, ratio=1)
fig4 + Points(scales(color=brewer("qual", "Pastel1")),size=3)
fig4

In [ ]:
fig5 = Figure(iris, 
              mapping(x="PetalLength", y="SepalWidth", color="Name"),
              width=256, ratio=1)
fig5 + Points()
fig5

In [ ]:
# from hcplot.scale import brewer
import numpy as np

def trans(array):
    return [[c[i] for c in array] for i in range(3)]

def rgbDF(typ, palette):
    colors = brewer(typ, palette, True)(-1)
    x = list(range(len(colors)))
    r, g, b = trans(colors)
    return pd.DataFrame({"X":x, "R":r, "G":g, "B":b})

def rgbiDF(typ, palette, size):
    xmax = len(brewer(typ, palette, True)(-1)) - 1
    x = np.linspace(0,1,size)
    colors = brewer(typ, palette, False)(x)
    r, g, b = trans(colors)
    return pd.DataFrame({"X":x*xmax, "R":r, "G":g, "B":b})

In [ ]:
typ, palette = "div", "PiYG"
data1 = rgbDF(typ, palette)
data2 = rgbiDF(typ, palette, 100)

fig = Figure(data1) \
    + Line(       mapping("X","R"), color="#ff0000", lineType="Dot", marker="triangle") \
    + Line(       mapping("X","G"), color="#00ff00", lineType="Dot", marker="triangle") \
    + Line(       mapping("X","B"), color="#0000ff", lineType="Dot", marker="triangle") \
    + Line(data2, mapping("X","R"), color="#ff0000", lineWidth=1,    showMarker=False)  \
    + Line(data2, mapping("X","G"), color="#00ff00", lineWidth=1,    showMarker=False)  \
    + Line(data2, mapping("X","B"), color="#0000ff", lineWidth=1,    showMarker=False)
fig

# BASE

# LAYER

# LINE

# FIGURE

# COLOR

# SCALE

# UTILS

# DATA

# TEMPLATES